когда то на разборе всероса и правда говорили про то что засунули готовые эмбедв в катбуст и заработало
и типа это сильно лучше

я бы попробовал все и заыигачил ансамбль


условно можно представить задачу текст картинка и табличные данные


я бы и клип зафигачил


и чисто по тексту попробовал предсказать


и чисто по картинке


и все в один засунул


и засунул бы без картинки в катбуст


крч как можно больше попытался перебрать


СРАВНИТЬ 

# Сиды

In [1]:
import numpy as np
import pandas as pd
import torch
import random
import os 

seed=42

np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)



In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Импорты

In [3]:
import timm
import transformers
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoProcessor, AutoModel, CLIPModel, CLIPProcessor
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import Dataset, DataLoader
import tqdm
from tqdm import tqdm 
from catboost import CatBoostClassifier
from torchvision import transforms as v2
from PIL import Image
from glob import glob
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

2025-09-22 07:23:53.293724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758525833.488220      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758525833.541383      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# TEST MODE

In [4]:
TEST_MODE=True

# EPOCHS

In [5]:
EPOCHS=1

# Загрузка данных

In [6]:
train=pd.read_csv('/kaggle/input/petfinder-adoption-prediction/train/train.csv')
test=pd.read_csv('/kaggle/input/petfinder-adoption-prediction/test/test.csv')
sample=pd.read_csv('/kaggle/input/petfinder-adoption-prediction/test/sample_submission.csv')

breed_labels=pd.read_csv('/kaggle/input/petfinder-adoption-prediction/breed_labels.csv')
color_labels=pd.read_csv('/kaggle/input/petfinder-adoption-prediction/color_labels.csv')
state_labels=pd.read_csv('/kaggle/input/petfinder-adoption-prediction/StateLabels.csv')


train_img_dir='/kaggle/input/petfinder-adoption-prediction/train_images'
test_img_dir='/kaggle/input/petfinder-adoption-prediction/test_images'

### Merge

In [7]:
# === 1) Подготовка справочников ===
breed_map = breed_labels.set_index('BreedID')['BreedName'].to_dict()
breed_type_map = breed_labels.set_index('BreedID')['Type'].to_dict()  # иногда полезно
color_map = color_labels.set_index('ColorID')['ColorName'].to_dict()
state_map = state_labels.set_index('StateID')['StateName'].to_dict()

# карта для Type (по описанию PetFinder: 1=Dog, 2=Cat)
type_map = {1: 'Dog', 2: 'Cat'}

def enrich_petfinder_df(df):
    df = df.copy()

    # --- Породы ---
    df['Breed1Name'] = df['Breed1'].map(breed_map)
    df['Breed2Name'] = df['Breed2'].map(breed_map)
    # полезно знать, совпадает ли тип породы с заявленным Type
    df['Breed1Type'] = df['Breed1'].map(breed_type_map)
    df['Breed2Type'] = df['Breed2'].map(breed_type_map)
    # индикаторы наличия второй породы и «смешанная порода»
    df['HasBreed2'] = (df['Breed2'] > 0).astype('int8')
    df['IsMixedBreed'] = (df['HasBreed2'] == 1).astype('int8')

    # агрегированное поле «порода»
    def _agg_breed(row):
        b1 = row['Breed1Name'] if pd.notna(row['Breed1Name']) else ''
        b2 = row['Breed2Name'] if pd.notna(row['Breed2Name']) else ''
        if b2 and b2 != b1:
            return f"{b1} + {b2}"
        return b1
    df['BreedAgg'] = df.apply(_agg_breed, axis=1)

    # --- Цвета ---
    df['Color1Name'] = df['Color1'].map(color_map)
    df['Color2Name'] = df['Color2'].map(color_map)
    df['Color3Name'] = df['Color3'].map(color_map)

    # агрегированный список цветов и их количество
    def _agg_colors(row):
        cols = [row['Color1Name'], row['Color2Name'], row['Color3Name']]
        cols = [c for c in cols if isinstance(c, str) and len(c)]
        # уберём дубли, сохранив порядок
        seen, out = set(), []
        for c in cols:
            if c not in seen:
                seen.add(c); out.append(c)
        return out
    df['ColorsList'] = df.apply(_agg_colors, axis=1)
    df['NumColors'] = df['ColorsList'].apply(len).astype('int8')
    # удобная строка цветов
    df['ColorsAgg'] = df['ColorsList'].apply(lambda xs: ' / '.join(xs) if xs else '')

    # --- Штаты/регионы ---
    df['StateName'] = df['State'].map(state_map)

    # --- Тип питомца строкой ---
    df['TypeName'] = df['Type'].map(type_map)

    return df

# Применяем к train/test
train_en = enrich_petfinder_df(train)
test_en  = enrich_petfinder_df(test)

# (необязательно) покажем, что всё сработало
cols_to_peek = [
    'PetID','Type','TypeName','Breed1','Breed1Name','Breed2','Breed2Name',
    'IsMixedBreed','Color1','Color1Name','Color2','Color2Name','Color3','Color3Name',
    'ColorsAgg','NumColors','State','StateName'
]
display(train_en[cols_to_peek].head())


,PetID,Type,TypeName,Breed1,Breed1Name,Breed2,Breed2Name,IsMixedBreed,Color1,Color1Name,Color2,Color2Name,Color3,Color3Name,ColorsAgg,NumColors,State,StateName
0,86e1089a3,2,Cat,299,Tabby,0,NaN,0,1,Black,7,White,0,NaN,Black / White,2,41326,Selangor
1,6296e909a,2,Cat,265,Domestic Medium Hair,0,NaN,0,1,Black,2,Brown,0,NaN,Black / Brown,2,41401,Kuala Lumpur
2,3422e4906,1,Dog,307,Mixed Breed,0,NaN,0,2,Brown,7,White,0,NaN,Brown / White,2,41326,Selangor
3,5842f1ff5,1,Dog,307,Mixed Breed,0,NaN,0,1,Black,2,Brown,0,NaN,Black / Brown,2,41401,Kuala Lumpur
4,850a43f90,1,Dog,307,Mixed Breed,0,NaN,0,1,Black,0,NaN,0,NaN,Black,1,41326,Selangor


# TEST

In [8]:
if TEST_MODE:
    train_en=train_en[:10000]
else:
    train_en=train_en

# Train  test split

In [9]:
tr_df, va_df = train_en.copy(), None
tr_df, va_df = train_test_split(
    train_en, test_size=0.2, random_state=seed, stratify=train_en["AdoptionSpeed"]
)

# Текст

In [10]:
TEXT_BACKBONE = "microsoft/deberta-v3-base" 

In [11]:
model_for_text=AutoModelForSequenceClassification.from_pretrained(TEXT_BACKBONE, num_labels=5)
tokenizer=AutoTokenizer.from_pretrained(TEXT_BACKBONE)

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


# Картинки

In [12]:
IMG_BACKBONE  = "vit_base_patch16_224"  

In [13]:
model_for_image=timm.create_model(IMG_BACKBONE, pretrained=True, num_classes=5,).to(device)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

# Датасет

In [14]:
"""
class PetFinderDataset1(Dataset):
    def __init__(self, df, img_dir, transform, train, tabular, tokenizer, is_text,  ):
        self.df=df
        self.img_dir=img_dir
        self.transform=transform
        self.train=train
        self.tabular=tabular
        self.tokenizer=tokenizer
        self.is_text=is_text
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        image_root=row['PetID']
        image_all=os.path.join(self.img_dir, f'{image_root}-1.jpg')
        image=Image.open(image_all).convert('RGB')
        if self.transform is not None:
            image=self.transform(image)
        else:
            image=image
        if self.is_text:
            text=row['Description']
            tokenized=self.tokenizer(
                text, 
                padding="max_length", 
                truncation=True, 
                max_length=256, 
                return_tensors="pt"
                
                
            )
        if self.tabular is not None:
            tab=torch.tensor(row[self.tabular].astype('float32').values, dtype=torch.float32)
        else:
            tab=None

        

        
        if self.train:
            label=torch.tensor(int(row['AdoptionSpeed']), dtype=torch.long)
            return({
                'image': image,
                'label': label,
                'input_ids' : tokenized["input_ids"].squeeze(0),
                'attention_mask': tokenized["attention_mask"].squeeze(0),
                'tabular': tab
            })
        else:
            return({
                'image': image, 
                '':
                'input_ids' : tokenized["input_ids"].squeeze(0),
                'attention_mask': tokenized["attention_mask"].squeeze(0),
                'tabular': tab
            })

"""

'\nclass PetFinderDataset1(Dataset):\n    def __init__(self, df, img_dir, transform, train, tabular, tokenizer, is_text,  ):\n        self.df=df\n        self.img_dir=img_dir\n        self.transform=transform\n        self.train=train\n        self.tabular=tabular\n        self.tokenizer=tokenizer\n        self.is_text=is_text\n    def __len__(self):\n        return len(self.df)\n    def __getitem__(self, idx):\n        row=self.df.iloc[idx]\n        image_root=row[\'PetID\']\n        image_all=os.path.join(self.img_dir, f\'{image_root}-1.jpg\')\n        image=Image.open(image_all).convert(\'RGB\')\n        if self.transform is not None:\n            image=self.transform(image)\n        else:\n            image=image\n        if self.is_text:\n            text=row[\'Description\']\n            tokenized=self.tokenizer(\n                text, \n                padding="max_length", \n                truncation=True, \n                max_length=256, \n                return_tensors="pt"

In [15]:
class PetFinderDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, tabular_cols=None,
                 tokenizer=None, is_train=True, max_len=256):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.tabular_cols = tabular_cols or []
        self.tokenizer = tokenizer
        self.is_train = is_train
        self.max_len = max_len

    def _load_image(self, pet_id):
        path1 = os.path.join(self.img_dir, f"{pet_id}-1.jpg")
        if os.path.exists(path1):
            img = Image.open(path1).convert("RGB")
        else:
            alts = sorted(glob(os.path.join(self.img_dir, f"{pet_id}-*.jpg")))
            img = Image.open(alts[0]).convert("RGB") if alts else Image.new("RGB",(256,256),(0,0,0))
        return self.transform(img) if self.transform else v2.ToTensor()(img)

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = self._load_image(row["PetID"])

        text = row["Description"] if isinstance(row["Description"], str) else ""
        toks = self.tokenizer(
            text, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt"
        )
        input_ids      = toks["input_ids"].squeeze(0)
        attention_mask = toks["attention_mask"].squeeze(0)

        tab = torch.tensor(row[self.tabular_cols].astype("float32").values, dtype=torch.float32) \
              if self.tabular_cols else torch.empty(0, dtype=torch.float32)

        item = {"image": image, "input_ids": input_ids, "attention_mask": attention_mask, "tabular": tab}
        if self.is_train:
            item["labels"] = torch.tensor(int(row["AdoptionSpeed"]), dtype=torch.long)
        return item

In [16]:
class TextOnlyDataset(Dataset):
    def __init__(self, df, tokenizer, is_train=True, max_len=256):
        self.df=df.reset_index(drop=True); self.tok=tokenizer
        self.is_train=is_train; self.max_len=max_len
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row=self.df.iloc[i]
        txt = row["Description"] if isinstance(row["Description"], str) else ""
        toks=self.tok(txt, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt")
        item = {k:v.squeeze(0) for k,v in toks.items()}
        if self.is_train:
            item["labels"] = torch.tensor(int(row["AdoptionSpeed"]), dtype=torch.long)
        return item

tr_text = TextOnlyDataset(tr_df, tokenizer, True)
va_text = TextOnlyDataset(va_df, tokenizer, True)

from sklearn.metrics import cohen_kappa_score, f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "macro_f1": f1_score(labels, preds, average="macro"),
        "qwk": cohen_kappa_score(labels, preds, weights="quadratic"),
    }

In [17]:
class CLIPDataset(Dataset):
    def __init__(self, df, img_dir, text_col="Description"):
        self.df=df
        self.img_dir=img_dir
        self.text_col=text_col
    def __len__(self):
        return len(self.df)
    def _pick_image(self, pet_id):
        # 1-я фотка, если нет — любая, если пусто — заглушка
        p1 = os.path.join(self.img_dir, f"{pet_id}-1.jpg")
        if os.path.exists(p1):
            return Image.open(p1).convert("RGB")
        alts = sorted(glob(os.path.join(self.img_dir, f"{pet_id}-*.jpg")))
        if alts:
            return Image.open(alts[0]).convert("RGB")
        # заглушка (редко, но лучше не падать)
        return Image.new("RGB", (256, 256), (0, 0, 0))
        
    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        img=self._pick_image(row["PetID"])
        txt = row.get(self.text_col, "")
        if pd.isna(txt): txt = ""
        return {"image_pil": img, "text": txt}

## Tabular feats

In [18]:
TAB_COLS = [
    'Type','Age','Breed1','Breed2','Gender','Color1','Color2','Color3',
    'MaturitySize','FurLength','Vaccinated','Dewormed','Sterilized','Health',
    'Quantity','Fee','VideoAmt','PhotoAmt','State'
]


In [19]:
categorical_feats=train[TAB_COLS].select_dtypes(include='object').columns.tolist()

## Аугментации

In [20]:
IMG_SIZE=224

In [21]:
IMG_SIZE=224
train_tfms = v2.Compose([
    v2.Resize((IMG_SIZE, IMG_SIZE)),
    v2.RandomHorizontalFlip(),
    v2.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.03),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
val_tfms = v2.Compose([
    v2.Resize((IMG_SIZE, IMG_SIZE)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

## Создание датасетов

In [22]:
train_ds = PetFinderDataset(tr_df,  train_img_dir, train_tfms, TAB_COLS, tokenizer, is_train=True)
val_ds   = PetFinderDataset(va_df,  train_img_dir, val_tfms,   TAB_COLS, tokenizer, is_train=True)
test_ds  = PetFinderDataset(test_en,test_img_dir, val_tfms,   TAB_COLS, tokenizer, is_train=False)

# loaders


## DataLoader

In [23]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

# CLIP

In [24]:
# now InfoNCE then cross entropy 
model_clip=CLIPModel.from_pretrained('openai/clip-vit-base-patch32', ).to(device)
clip_processor=CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [25]:
def clip_collate_fn(batch):
    images = [b["image_pil"] for b in batch]
    texts  = [b["text"]     for b in batch]
    proc = clip_processor(text=texts, images=images, padding=True, truncation=True, return_tensors="pt")
    # вернём CPU тензоры; перевод на GPU сделаем в train/eval лупах
    return proc

In [26]:
clip_train_ds = CLIPDataset(tr_df, img_dir=train_img_dir, text_col="Description")
clip_val_ds   = CLIPDataset(va_df, img_dir=train_img_dir, text_col="Description")

clip_train_loader = DataLoader(clip_train_ds, batch_size=64, shuffle=True,
                               num_workers=2, pin_memory=True, collate_fn=clip_collate_fn)
clip_val_loader   = DataLoader(clip_val_ds,   batch_size=64, shuffle=False,
                               num_workers=2, pin_memory=True, collate_fn=clip_collate_fn)

In [27]:
'''
from torch.optim import AdamW
from tqdm import tqdm

for p in model_clip.parameters(): 
    p.requires_grad = True  # при желании можно частично разморозить

opt = AdamW(model_clip.parameters(), lr=1e-5, weight_decay=0.02)

def train_clip_one_epoch(model, loader, optimizer):
    model.train()
    running = 0.0; n = 0
    pbar = tqdm(loader, desc="CLIP train", leave=False)
    for batch in pbar:
        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        out = model(**batch, return_loss=True)   # считает 0.5*(i2t + t2i)
        loss = out.loss
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        bs = batch["input_ids"].size(0)
        running += loss.item() * bs; n += bs
        pbar.set_postfix(loss=running / max(n,1))
    return running / max(n,1)

for epoch in range(1, EPOCHS+1):
    tr_loss = train_clip_one_epoch(model_clip, clip_train_loader, opt, )
    print(f"[epoch {epoch}] train_loss={tr_loss:.4f}")
'''

'\nfrom torch.optim import AdamW\nfrom tqdm import tqdm\n\nfor p in model_clip.parameters(): \n    p.requires_grad = True  # при желании можно частично разморозить\n\nopt = AdamW(model_clip.parameters(), lr=1e-5, weight_decay=0.02)\n\ndef train_clip_one_epoch(model, loader, optimizer):\n    model.train()\n    running = 0.0; n = 0\n    pbar = tqdm(loader, desc="CLIP train", leave=False)\n    for batch in pbar:\n        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}\n        out = model(**batch, return_loss=True)   # считает 0.5*(i2t + t2i)\n        loss = out.loss\n        optimizer.zero_grad(set_to_none=True)\n        loss.backward()\n        optimizer.step()\n        bs = batch["input_ids"].size(0)\n        running += loss.item() * bs; n += bs\n        pbar.set_postfix(loss=running / max(n,1))\n    return running / max(n,1)\n\nfor epoch in range(1, EPOCHS+1):\n    tr_loss = train_clip_one_epoch(model_clip, clip_train_loader, opt, )\n    print(f"[epoch {epoch}] 

In [28]:
from transformers import CLIPModel, CLIPProcessor

#model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
#clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

for p in model_clip.parameters():
    p.requires_grad = False
model_clip.eval()  # важно


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [29]:
import torch, numpy as np
import torch.nn.functional as F

@torch.no_grad()
def extract_clip_features(model, loader):
    model.eval()
    all_img, all_txt = [], []
    for batch in tqdm(loader, desc="CLIP feats (val)", leave=False):
        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        img_f = model.get_image_features(pixel_values=batch["pixel_values"])   # (B,D)
        txt_f = model.get_text_features(input_ids=batch["input_ids"],
                                        attention_mask=batch["attention_mask"])
        img_f = F.normalize(img_f, dim=-1)
        txt_f = F.normalize(txt_f, dim=-1)
        all_img.append(img_f.cpu()); all_txt.append(txt_f.cpu())
    all_img = torch.cat(all_img, dim=0).numpy()
    all_txt = torch.cat(all_txt, dim=0).numpy()
    return all_img, all_txt  # (N,D), (N,D)

def recall_at_k(sim, k):
    # sim: (N,N), строки — img, столбцы — txt; правильная пара всегда по диагонали
    idx_topk = np.argpartition(-sim, kth=k-1, axis=1)[:, :k]  # индексы топ-k по строкам
    hits = np.any(idx_topk == np.arange(sim.shape[0])[:, None], axis=1)
    return float(hits.mean())

@torch.no_grad()
def evaluate_clip_retrieval(model, loader):
    img, txt = extract_clip_features(model, loader)
    sim = img @ txt.T
    r1  = recall_at_k(sim, 1)
    r5  = recall_at_k(sim, 5)
    r10 = recall_at_k(sim, 10)
    # симметрично для t2i
    sim_t = sim.T
    r1_t  = recall_at_k(sim_t, 1)
    r5_t  = recall_at_k(sim_t, 5)
    r10_t = recall_at_k(sim_t, 10)
    return {
        "i2t_R@1": r1, "i2t_R@5": r5, "i2t_R@10": r10,
        "t2i_R@1": r1_t, "t2i_R@5": r5_t, "t2i_R@10": r10_t
    }

metrics = evaluate_clip_retrieval(model_clip, clip_val_loader)
print(metrics)


{'i2t_R@1': 0.0135, 'i2t_R@5': 0.0355, 'i2t_R@10': 0.056, 't2i_R@1': 0.0165, 't2i_R@5': 0.043, 't2i_R@10': 0.067}


# Просто по тексту

In [30]:
use_fp16 = (device.type == "cuda")

args = TrainingArguments(
    output_dir="only_text",
    # батчи и аккумулирование — чтобы держать эффективный BS≈32 даже если GPU слабее
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,

    num_train_epochs=1, #5
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,                  # ~10% разогрева почти всегда помогает

    # более стабильное обучение
    max_grad_norm=1.0,
    label_smoothing_factor=0.1,        # слегка сглаживаем таргеты (полезно при дисбалансе)
    gradient_checkpointing=True,       # экономим память (немного медленнее)

    # валидация/сохранения/лучшая модель
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="qwk",
    greater_is_better=True,

    # ускорение
    fp16=use_fp16,
    bf16=False,                        # на T4/Каггле bf16 обычно нет

    # логирование и детерминизм
    logging_steps=50,
    report_to="none",
    seed=42,

    # загрузчики
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
)

# ——— МЕТРИКИ: accuracy, macro/weighted F1 и Quadratic Weighted Kappa ———
def compute_metrics(eval_pred):
    """
    eval_pred: (predictions, label_ids) от Trainer
    predictions — либо logits, либо tuple(logits, ...).
    """
    preds, labels = eval_pred
    if isinstance(preds, (list, tuple)):
        logits = preds[0]
    else:
        logits = preds
    if isinstance(logits, torch.Tensor):
        logits = logits.detach().cpu().numpy()

    y_true = labels
    y_pred = logits.argmax(axis=-1)

    acc  = accuracy_score(y_true, y_pred)
    f1_m = f1_score(y_true, y_pred, average="macro")
    f1_w = f1_score(y_true, y_pred, average="weighted")
    qwk  = cohen_kappa_score(y_true, y_pred, weights="quadratic")

    return {
        "accuracy": acc,
        "macro_f1": f1_m,
        "weighted_f1": f1_w,
        "qwk": qwk,
    }


In [31]:
trainer=Trainer(
    args=args, 
    model=model_for_text,
    train_dataset=tr_text,
    eval_dataset=va_text,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Macro F1,Weighted F1,Qwk
200,1.487200,1.479099,0.327000,0.171833,0.235324,0.067368


TrainOutput(global_step=250, training_loss=1.5016693420410157, metrics={'train_runtime': 354.6645, 'train_samples_per_second': 22.557, 'train_steps_per_second': 0.705, 'total_flos': 1052491444224000.0, 'train_loss': 1.5016693420410157, 'epoch': 1.0})

# Просто по картинке

In [33]:

criterion=torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_for_image.parameters(), lr=3e-4, weight_decay=0.05)
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [34]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [35]:
'''
for epoch in range(1, EPOCHS+1):
    model_for_image.train()
    pbar=tqdm(train_loader, desc='training', leave=False)
    running_loss, num_correct, n=0.0,0,0

    for step, batch in enumerate(pbar):
        optimizer.zero_grad()
        X=batch['image'].to(device)
        y=batch['labels'].to(device)

        logits=model_for_image(X)
        loss=criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()*X.size(0)
        pbar.set_postfix(loss=running_loss/((step+1)*X.size(0)))
    scheduler.step()
    print(f'train: loss={running_loss/n:.4f}')
    model_for_image.eval()
    
with torch.no_grad():
    pbar=tqdm(val_loader, desc='evaluation', leave=False)
    loss_sum, correct, n=0.0,0,0

    all_probs, all_targets=[],[]
    for batch in  pbar:
        X=batch['image'].to(device)
        y=batch['labels'].to(device)

        logits=model_for_image(X)

        loss=criterion(logits, y)
        loss_sum+=loss.item()*X.size(0)

        n+=X.size(0)
        probs=torch.softmax(logits, dim=1).detach().cpu().numpy()
        all_probs.append(probs)
        all_targets.append(y.detach().cpu().numpy())
    all_probs=np.concatenate(all_probs, axis=0)
    all_targets=np.concatenate(all_targets, axis=0)
    print(f"valid: loss={val_loss:.4f}")
    
    '''

'\nfor epoch in range(1, EPOCHS+1):\n    model_for_image.train()\n    pbar=tqdm(train_loader, desc=\'training\', leave=False)\n    running_loss, num_correct, n=0.0,0,0\n\n    for step, batch in enumerate(pbar):\n        optimizer.zero_grad()\n        X=batch[\'image\'].to(device)\n        y=batch[\'labels\'].to(device)\n\n        logits=model_for_image(X)\n        loss=criterion(logits, y)\n        loss.backward()\n        optimizer.step()\n\n        running_loss+=loss.item()*X.size(0)\n        pbar.set_postfix(loss=running_loss/((step+1)*X.size(0)))\n    scheduler.step()\n    print(f\'train: loss={running_loss/n:.4f}\')\n    model_for_image.eval()\n    \nwith torch.no_grad():\n    pbar=tqdm(val_loader, desc=\'evaluation\', leave=False)\n    loss_sum, correct, n=0.0,0,0\n\n    all_probs, all_targets=[],[]\n    for batch in  pbar:\n        X=batch[\'image\'].to(device)\n        y=batch[\'labels\'].to(device)\n\n        logits=model_for_image(X)\n\n        loss=criterion(logits, y)\n    

In [36]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

# Критерий с label smoothing — часто помогает на адопшн-рангах
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

use_amp = (device.type == "cuda")
scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)

def evaluate(model, loader, criterion):
    model.eval()
    loss_sum, n = 0.0, 0
    all_probs, all_targets = [], []

    with torch.no_grad():
        pbar = tqdm(loader, desc="validation", leave=False)
        for batch in pbar:
            X = batch["image"].to(device, non_blocking=True)
            y = batch["labels"].to(device, non_blocking=True)

            with torch.cuda.amp.autocast(enabled=use_amp):
                logits = model(X)
                loss   = criterion(logits, y)

            bs = X.size(0)
            loss_sum += loss.item() * bs
            n        += bs

            probs = torch.softmax(logits, dim=1).float().cpu().numpy()
            all_probs.append(probs)
            all_targets.append(y.detach().cpu().numpy())

    val_loss   = loss_sum / max(n, 1)
    probs      = np.concatenate(all_probs, axis=0)
    targets    = np.concatenate(all_targets, axis=0)
    preds      = probs.argmax(axis=1)

    acc        = accuracy_score(targets, preds)
    macro_f1   = f1_score(targets, preds, average="macro")
    qwk        = cohen_kappa_score(targets, preds, weights="quadratic")

    metrics = {"acc": acc, "macro_f1": macro_f1, "qwk": qwk}
    return val_loss, probs, targets, metrics

# --------- ТРЕНИРОВКА С ВАЛИДАЦИЕЙ, КЛИППИНГОМ И EARLY STOP ---------
best_qwk   = -1.0
patience   = 3   # остановимся, если QWK не растёт 'patience' эпох подряд
wait       = 0

for epoch in range(1, EPOCHS + 1):
    model_for_image.train()
    running_loss, n = 0.0, 0
    pbar = tqdm(train_loader, desc=f"training epoch {epoch}", leave=False)

    for step, batch in enumerate(pbar):
        X = batch["image"].to(device, non_blocking=True)
        y = batch["labels"].to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=use_amp):
            logits = model_for_image(X)
            loss   = criterion(logits, y)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model_for_image.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        bs = X.size(0)
        running_loss += loss.item() * bs
        n            += bs
        pbar.set_postfix(train_loss=running_loss / max(n, 1))

    train_loss = running_loss / max(n, 1)

    # — валидация —
    val_loss, val_probs, val_targets, val_metrics = evaluate(model_for_image, val_loader, criterion)

    # планировщик после эпохи
    try:
        scheduler.step()
    except Exception:
        pass

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f} | "
        f"val_loss={val_loss:.4f} | "
        f"acc={val_metrics['acc']:.4f} | "
        f"macro_f1={val_metrics['macro_f1']:.4f} | "
        f"qwk={val_metrics['qwk']:.4f}"
    )

    # — сохраняем лучшую по QWK —
    if val_metrics["qwk"] > best_qwk:
        best_qwk = val_metrics["qwk"]
        wait = 0
        torch.save(
            {
                "model_state": model_for_image.state_dict(),
                "epoch": epoch,
                "val_metrics": val_metrics,
            },
            "best_vit_qwk.pt",
        )
        print(f"✓ new best QWK={best_qwk:.4f} — checkpoint saved to best_vit_qwk.pt")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping: no QWK improvement.")
            break


/tmp/ipykernel_19/3887674590.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)
training epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]/tmp/ipykernel_19/3887674590.py:60: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):
validation:   0%|          | 0/32 [00:00<?, ?it/s]/tmp/ipykernel_19/3887674590.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


Epoch 01 | train_loss=1.5637 | val_loss=1.5045 | acc=0.2840 | macro_f1=0.0903 | qwk=0.0026
✓ new best QWK=0.0026 — checkpoint saved to best_vit_qwk.pt


# Экстрактим фичи из всех моделей 

In [37]:
from transformers import AutoModel

# вытаскиваем encoder из обученного классификатора
from transformers import AutoModel

# берём базовый encoder той же архитектуры
text_encoder = AutoModel.from_pretrained(TEXT_BACKBONE).to(device).eval()

# подгружаем веса base из твоего дообученного классификатора
if hasattr(model_for_text, "deberta"):
    base_state = model_for_text.deberta.state_dict()
elif hasattr(model_for_text, "bert"):          # на всякий случай для совместимости
    base_state = model_for_text.bert.state_dict()
else:
    # универсально: многие HF-классы имеют .base_model
    base_state = model_for_text.base_model.state_dict()

text_encoder.load_state_dict(base_state)

@torch.no_grad()
def embed_texts(texts, tokenizer, model, batch_size=256, max_len=256):
    out_all = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        toks = tokenizer(list(chunk), padding=True, truncation=True,
                         max_length=max_len, return_tensors="pt")
        toks = {k: v.to(device) for k, v in toks.items()}
        out = model(**toks)                       # BaseModel output
        cls = out.last_hidden_state[:, 0, :]      # [CLS]/первый токен
        out_all.append(cls.float().cpu().numpy())
    return np.vstack(out_all).astype(np.float32)

tr_text_emb   = embed_texts(tr_df.Description.fillna("").tolist(), tokenizer, text_encoder)
va_text_emb   = embed_texts(va_df.Description.fillna("").tolist(), tokenizer, text_encoder)
test_text_emb = embed_texts(test_en.Description.fillna("").tolist(), tokenizer, text_encoder)

In [38]:
# test CLIP dataloader
clip_test_ds = CLIPDataset(test_en, img_dir=test_img_dir, text_col="Description")
clip_test_loader = DataLoader(clip_test_ds, batch_size=64, shuffle=False,
                              num_workers=2, pin_memory=True, collate_fn=clip_collate_fn)


In [39]:
import torch.nn.functional as F

@torch.no_grad()
def clip_image_embeddings(model, loader):
    model.eval()
    feats = []
    for batch in tqdm(loader, desc="CLIP img emb", leave=False):
        pixel_values = batch["pixel_values"].to(
            device, dtype=next(model.parameters()).dtype, non_blocking=True
        )
        f = model.get_image_features(pixel_values=pixel_values)
        f = F.normalize(f, dim=-1)
        feats.append(f.float().cpu().numpy())
    return np.vstack(feats).astype(np.float32)

@torch.no_grad()
def clip_text_embeddings(model, loader):
    model.eval()
    feats = []
    for batch in tqdm(loader, desc="CLIP txt emb", leave=False):
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        f = model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        f = F.normalize(f, dim=-1)
        feats.append(f.float().cpu().numpy())
    return np.vstack(feats).astype(np.float32)

# извлекаем
clip_tr_ld = DataLoader(clip_train_ds, batch_size=64, shuffle=False, num_workers=2,
                        pin_memory=True, collate_fn=clip_collate_fn)
clip_va_ld = clip_val_loader
clip_te_ld = clip_test_loader

tr_clip_img = clip_image_embeddings(model_clip, clip_tr_ld)   # (Ntr, 512)
va_clip_img = clip_image_embeddings(model_clip, clip_va_ld)   # (Nva, 512)
te_clip_img = clip_image_embeddings(model_clip, clip_te_ld)   # (Nte, 512)

# опционально: текстовые эмбеддинги CLIP
tr_clip_txt = clip_text_embeddings(model_clip, clip_tr_ld)    # (Ntr, 512)
va_clip_txt = clip_text_embeddings(model_clip, clip_va_ld)    # (Nva, 512)
te_clip_txt = clip_text_embeddings(model_clip, clip_te_ld)    # (Nte, 512)


In [40]:
@torch.no_grad()
@torch.no_grad()
def timm_image_embeddings(model, loader, use_pre_logits=True):
    model.eval()
    feats = []
    for batch in tqdm(loader, desc="timm img emb", leave=False):
        x = batch["image"].to(device, non_blocking=True)

        f = model.forward_features(x)
        # Предпочитаемый путь для timm (работает и для ViT, и для CNN):
        try:
            f = model.forward_head(f, pre_logits=use_pre_logits)  # -> (B, D)
        except Exception:
            # Фоллбек на случай нестандартной головы/модели
            if isinstance(f, dict):
                f = f.get("x", f.get("pooled", next(iter(f.values()))))
            if isinstance(f, (list, tuple)):
                f = f[-1]
            if f.ndim == 4:          # (B, C, H, W)
                f = f.mean(dim=(2, 3))
            elif f.ndim == 3:        # (B, N, D) — ViT-токены
                # берём CLS-токен, если он первый; иначе усредним по токенам
                f = f[:, 0, :] if f.size(1) >= 1 else f.mean(dim=1)
            elif f.ndim == 2:        # (B, D)
                pass
            else:                    # на всякий
                f = f.view(f.size(0), -1)

        feats.append(f.float().cpu().numpy())

    return np.vstack(feats).astype(np.float32)


# лоадер для test (если ещё не сделали)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

tr_img_emb = timm_image_embeddings(model_for_image, train_loader)  # (Ntr, D_vit≈768)
va_img_emb = timm_image_embeddings(model_for_image, val_loader)    # (Nva, D_vit≈768)
te_img_emb = timm_image_embeddings(model_for_image, test_loader)   # (Nte, D_vit≈768)


In [41]:
TAB_COLS = [
    'Type','Age','Breed1','Breed2','Gender','Color1','Color2','Color3',
    'MaturitySize','FurLength','Vaccinated','Dewormed','Sterilized','Health',
    'Quantity','Fee','VideoAmt','PhotoAmt','State'
]

Xtr_tab = tr_df[TAB_COLS].astype(np.float32).values
Xva_tab = va_df[TAB_COLS].astype(np.float32).values
Xte_tab = test_en[TAB_COLS].astype(np.float32).values

y_tr = tr_df["AdoptionSpeed"].astype(int).values
y_va = va_df["AdoptionSpeed"].astype(int).values


# Катбуст по исключительно табличным

In [42]:
from catboost import CatBoostClassifier

cbc_tab = CatBoostClassifier(
    iterations=1000 if not TEST_MODE else 200,
    learning_rate=0.03,
    depth=6,
    loss_function='MultiClass',
    eval_metric='TotalF1',
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    verbose=100
)
cbc_tab.fit(Xtr_tab, y_tr, eval_set=(Xva_tab, y_va))
probs_va_tab = cbc_tab.predict_proba(Xva_tab)


0:	learn: 0.3397399	test: 0.3314514	best: 0.3314514 (0)	total: 13s	remaining: 43m 4s
100:	learn: 0.4142077	test: 0.3649927	best: 0.3662710 (99)	total: 13.6s	remaining: 13.3s
199:	learn: 0.4467716	test: 0.3699944	best: 0.3704074 (154)	total: 14.2s	remaining: 0us
bestTest = 0.3704074473
bestIteration = 154
Shrink model to first 155 iterations.


In [43]:
#data for catboost


In [44]:
#model_catboost.fit()


In [45]:
#model_catboost.predict_proba()

# Эмбеддинги текста в катбуст

In [46]:
# ТОЛЬКО текстовые эмбеддинги (BERT CLS)
cbc_text = CatBoostClassifier(
    iterations=600 if not TEST_MODE else 200,
    learning_rate=0.05, depth=6,
    loss_function='MultiClass',
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    verbose=100
)
cbc_text.fit(tr_text_emb, y_tr, eval_set=(va_text_emb, y_va))
probs_va_text = cbc_text.predict_proba(va_text_emb)

# ТОЛЬКО img эмбеддинги от CLIP
cbc_img_clip = CatBoostClassifier(
    iterations=600 if not TEST_MODE else 200,
    learning_rate=0.05, depth=6,
    loss_function='MultiClass',
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    verbose=100
)
cbc_img_clip.fit(tr_clip_img, y_tr, eval_set=(va_clip_img, y_va))
probs_va_img_clip = cbc_img_clip.predict_proba(va_clip_img)

# ТОЛЬКО img эмбеддинги от timm (ResNet50)
cbc_img_timm = CatBoostClassifier(
    iterations=600 if not TEST_MODE else 200,
    learning_rate=0.05, depth=6,
    loss_function='MultiClass',
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    verbose=100
)
cbc_img_timm.fit(tr_img_emb, y_tr, eval_set=(va_img_emb, y_va))
probs_va_img_timm = cbc_img_timm.predict_proba(va_img_emb)

# ФЬЮЖН: табличка + текст (BERT) + картинка (CLIP image)
Xtr_fused = np.hstack([Xtr_tab, tr_text_emb, tr_clip_img])   # (Ntr, d_tab + 768 + 512)
Xva_fused = np.hstack([Xva_tab, va_text_emb, va_clip_img])

cbc_fused = CatBoostClassifier(
    iterations=800 if not TEST_MODE else 200,
    learning_rate=0.05, depth=6,
    loss_function='MultiClass',
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    verbose=100
)
cbc_fused.fit(Xtr_fused, y_tr, eval_set=(Xva_fused, y_va))
probs_va_fused = cbc_fused.predict_proba(Xva_fused)


0:	learn: 1.5932996	test: 1.5953774	best: 1.5953774 (0)	total: 16.3ms	remaining: 3.25s
100:	learn: 1.2905553	test: 1.4164861	best: 1.4164861 (100)	total: 1.35s	remaining: 1.32s
199:	learn: 1.1979349	test: 1.4112429	best: 1.4112429 (199)	total: 2.64s	remaining: 0us
bestTest = 1.41124292
bestIteration = 199


0:	learn: 1.5931639	test: 1.5945867	best: 1.5945867 (0)	total: 14.4ms	remaining: 2.86s
100:	learn: 1.2667675	test: 1.3911055	best: 1.3911055 (100)	total: 1.08s	remaining: 1.06s
199:	learn: 1.1653105	test: 1.3854779	best: 1.3854779 (199)	total: 2.11s	remaining: 0us
bestTest = 1.385477905
bestIteration = 199


0:	learn: 1.5988588	test: 1.5996936	best: 1.5996936 (0)	total: 16.7ms	remaining: 3.33s
100:	learn: 1.3738331	test: 1.4732848	best: 1.4719672 (77)	total: 1.32s	remaining: 1.29s
199:	learn: 1.2902837	test: 1.4792922	best: 1.4719672 (77)	total: 2.61s	remaining: 0us
bestTest = 1.471967163
bestIteration = 77
Shrink model to first 78 iterations.


0:	learn: 1.5891547	test: 1.5913633	best: 1.5913633 (0)	total: 25.5ms	remaining: 5.08s
100:	learn: 1.2070033	test: 1.3448085	best: 1.3448085 (100)	total: 2.02s	remaining: 1.99s
199:	learn: 1.0967271	test: 1.3364810	best: 1.3364810 (199)	total: 3.96s	remaining: 0us
bestTest = 1.336480957
bestIteration = 199


# Эмбеддинги картинки в катбуст

In [47]:
from sklearn.metrics import cohen_kappa_score, f1_score

def qwk_from_probs(y_true, *probs_list, weights=None):
    P = np.stack(probs_list, axis=-1)                # (N, 5, M)
    if weights is None:
        weights = np.ones(P.shape[-1], dtype=np.float32) / P.shape[-1]
    ens = (P * weights.reshape(1,1,-1)).sum(-1)      # (N,5)
    preds = ens.argmax(1)
    return cohen_kappa_score(y_true, preds, weights='quadratic'), preds

qwk_tab, _       = qwk_from_probs(y_va, probs_va_tab)
qwk_text, _      = qwk_from_probs(y_va, probs_va_text)
qwk_img_clip, _  = qwk_from_probs(y_va, probs_va_img_clip)
qwk_img_timm, _  = qwk_from_probs(y_va, probs_va_img_timm)
qwk_fused, _     = qwk_from_probs(y_va, probs_va_fused)

# простой ансамбль нескольких
w = np.array([0.2, 0.2, 0.2, 0.4])  # tab, text, img_clip, fused  (подбери по вал)
qwk_ens, preds_ens = qwk_from_probs(y_va, probs_va_tab, probs_va_text, probs_va_img_clip, probs_va_fused, weights=w)

print({
    "qwk_tab": qwk_tab, "qwk_text": qwk_text,
    "qwk_img_clip": qwk_img_clip, "qwk_img_timm": qwk_img_timm,
    "qwk_fused": qwk_fused, "qwk_ens": qwk_ens
})


{'qwk_tab': 0.3283265691644607, 'qwk_text': 0.10471546808008392, 'qwk_img_clip': 0.27950036565402236, 'qwk_img_timm': 0.0023521722469654893, 'qwk_fused': 0.32053137101387197, 'qwk_ens': 0.3015512865238119}


# Эмбеддинги текста и картинки в катбуст

In [48]:
#model_catboost_text_img=CatBoostClassifier(
    
#)

# Submission

In [49]:
#need inference notebook
# proba на тесте от базовых моделей
probs_te_tab     = cbc_tab.predict_proba(Xte_tab)
probs_te_text    = cbc_text.predict_proba(test_text_emb)
probs_te_imgclip = cbc_img_clip.predict_proba(te_clip_img)

# фьюжн на тесте
Xte_fused = np.hstack([Xte_tab, test_text_emb, te_clip_img])
probs_te_fused = cbc_fused.predict_proba(Xte_fused)

# ансамбль теми же весами w
Pte = np.stack([probs_te_tab, probs_te_text, probs_te_imgclip, probs_te_fused], axis=-1)
probs_te_ens = (Pte * w.reshape(1,1,-1)).sum(-1)
preds_test = probs_te_ens.argmax(1)

submission = sample.copy()
submission["AdoptionSpeed"] = preds_test
submission.to_csv("submission.csv", index=False)
print("submission.csv saved")


submission.csv saved


In [50]:
# ==== SAVE ALL ARTIFACTS ====
import os, json, torch, shutil
from pathlib import Path

ART_DIR = Path("artifacts")
ART_DIR.mkdir(parents=True, exist_ok=True)

# 1) DeBERTa classifier + tokenizer (лучший уже загружен в model_for_text благодаря Trainer(..., load_best_model_at_end=True))
TXT_DIR = ART_DIR / "deberta_cls"
TXT_DIR.mkdir(exist_ok=True)
model_for_text.save_pretrained(str(TXT_DIR))
tokenizer.save_pretrained(str(TXT_DIR))

# 2) DeBERTa text encoder (тот, что ты использовал для эмбеддингов)
TXT_ENC_DIR = ART_DIR / "deberta_text_encoder"
TXT_ENC_DIR.mkdir(exist_ok=True)
text_encoder.save_pretrained(str(TXT_ENC_DIR))

# 3) ViT classifier
IMG_DIR = ART_DIR / "vit_cls"
IMG_DIR.mkdir(exist_ok=True)

# текущее состояние (на случай если best_vit_qwk.pt не создан или хочешь еще и «текущее»)
torch.save(model_for_image.state_dict(), IMG_DIR / "vit_classifier_state.pth")

# если в тренинге сохранялся лучший чекпоинт — скопируем его вместе
best_ckpt_path = Path("best_vit_qwk.pt")
if best_ckpt_path.exists():
    shutil.copy2(best_ckpt_path, IMG_DIR / "best_vit_qwk.pt")

# 4) CatBoost models
cbc_tab.save_model(str(ART_DIR / "cbc_tab.cbm"))
cbc_text.save_model(str(ART_DIR / "cbc_text.cbm"))
cbc_img_clip.save_model(str(ART_DIR / "cbc_img_clip.cbm"))
cbc_img_timm.save_model(str(ART_DIR / "cbc_img_timm.cbm"))
cbc_fused.save_model(str(ART_DIR / "cbc_fused.cbm"))

# 5) Метаданные: чтобы восстановить pipeline без боли
meta = {
    "TEXT_BACKBONE": TEXT_BACKBONE,         # "microsoft/deberta-v3-base"
    "IMG_BACKBONE":  IMG_BACKBONE,          # "vit_base_patch16_224"
    "CLIP_BACKBONE": "openai/clip-vit-base-patch32",
    "num_classes": 5,
    "TAB_COLS": TAB_COLS,
    "ensemble_weights": list(map(float, w)),   # [0.2, 0.2, 0.2, 0.4]
    "image_norm_mean": [0.485, 0.456, 0.406],
    "image_norm_std":  [0.229, 0.224, 0.225],
    "img_size": 224,
    "seed": seed,
}
with open(ART_DIR / "metadata.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("✅ All artifacts saved to:", ART_DIR.resolve())


✅ All artifacts saved to: /kaggle/working/artifacts
